In [1]:
import scglue
from scglue.models.nn import GraphConv # import Graph convolution
from scglue.models.prob import ZILN, ZIN, ZINB
from typing import Any, List, Mapping, NoReturn, Optional, Tuple
from abc import abstractmethod

In [2]:
import torch
import torch.distributions as D
import torch.nn.functional as F
from scglue.num import EPS
# from scglue.models.sc import sc
import scglue.models.glue as glue

class DataDecoder(glue.DataDecoder):

    r"""
    Abstract data decoder

    Parameters
    ----------
    out_features
        Output dimensionality
    n_batches
        Number of batches
    """

    def __init__(self, out_features: int, n_batches: int = 1) -> None:  # pylint: disable=unused-argument
        super().__init__()

    @abstractmethod
    def forward(  # pylint: disable=arguments-differ
            self, u: torch.Tensor, v: torch.Tensor,
            b: torch.Tensor, l: Optional[torch.Tensor]
    ) -> D.Normal:
        r"""
        Decode data from sample and feature latent

        Parameters
        ----------
        u
            Sample latent
        v
            Feature latent
        b
            Batch index
        l
            Optional normalizer

        Returns
        -------
        recon
            Data reconstruction distribution
        """
        raise NotImplementedError  # pragma: no cover
        
class NBDataDecoder(DataDecoder):

    r"""
    Negative binomial data decoder

    Parameters
    ----------
    out_features
        Output dimensionality
    n_batches
        Number of batches
    """

    def __init__(self, out_features: int, n_batches: int = 1) -> None:
        super().__init__(out_features, n_batches)
        self.scale_lin = torch.nn.Parameter(torch.zeros(n_batches, out_features))
        self.bias = torch.nn.Parameter(torch.zeros(n_batches, out_features))
        self.log_theta = torch.nn.Parameter(torch.zeros(n_batches, out_features))

    def forward(
            self, u: torch.Tensor, v: torch.Tensor,
            b: torch.Tensor, l: torch.Tensor
    ) -> D.NegativeBinomial:
        #change [b] to [:,b]
#         scale = F.softplus(self.scale_lin[b])
        scale = F.softplus(self.scale_lin[:,b])
        print(u.shape,v.t().shape,scale.shape)
#         logit_mu = scale * (u @ v.t()) + self.bias[b]
        logit_mu = scale * (u @ v.t()) + self.bias[:,b]
        mu = F.softmax(logit_mu, dim=1) * l
        log_theta = self.log_theta[:,b]
        return D.NegativeBinomial(
            log_theta.exp(),
            logits=(mu + EPS).log() - log_theta
        )


In [3]:


class DataEncoder(glue.DataEncoder):

    r"""
    Abstract data encoder

    Parameters
    ----------
    in_features
        Input dimensionality
    out_features
        Output dimensionality
    h_depth
        Hidden layer depth
    h_dim
        Hidden layer dimensionality
    dropout
        Dropout rate
    """

    def __init__(
            self, in_features: int, out_features: int,
            h_depth: int = 2, h_dim: int = 256,
            dropout: float = 0.2
    ) -> None:
        super().__init__()
        self.h_depth = h_depth
        ptr_dim = in_features
        for layer in range(self.h_depth):
            setattr(self, f"linear_{layer}", torch.nn.Linear(ptr_dim, h_dim))
            setattr(self, f"act_{layer}", torch.nn.LeakyReLU(negative_slope=0.2))
            setattr(self, f"bn_{layer}", torch.nn.BatchNorm1d(h_dim))
            setattr(self, f"dropout_{layer}", torch.nn.Dropout(p=dropout))
            ptr_dim = h_dim
        self.loc = torch.nn.Linear(ptr_dim, out_features)
        self.std_lin = torch.nn.Linear(ptr_dim, out_features)

    @abstractmethod
    def compute_l(self, x: torch.Tensor) -> Optional[torch.Tensor]:
        r"""
        Compute normalizer

        Parameters
        ----------
        x
            Input data

        Returns
        -------
        l
            Normalizer
        """
        raise NotImplementedError  # pragma: no cover

    @abstractmethod
    def normalize(
            self, x: torch.Tensor, l: Optional[torch.Tensor]
    ) -> torch.Tensor:
        r"""
        Normalize data

        Parameters
        ----------
        x
            Input data
        l
            Normalizer

        Returns
        -------
        xnorm
            Normalized data
        """
        raise NotImplementedError  # pragma: no cover

    def forward(  # pylint: disable=arguments-differ
            self, x: torch.Tensor, xrep: torch.Tensor,
            lazy_normalizer: bool = True
    ) -> Tuple[D.Normal, Optional[torch.Tensor]]:
        r"""
        Encode data to sample latent distribution

        Parameters
        ----------
        x
            Input data
        xrep
            Alternative input data
        lazy_normalizer
            Whether to skip computing `x` normalizer (just return None)
            if `xrep` is non-empty

        Returns
        -------
        u
            Sample latent distribution
        normalizer
            Data normalizer

        Note
        ----
        Normalization is always computed on `x`.
        If xrep is empty, the normalized `x` will be used as input
        to the encoder neural network, otherwise xrep is used instead.
        """
        if xrep.numel():
            l = None if lazy_normalizer else self.compute_l(x)
            ptr = xrep
        else:
            l = self.compute_l(x)
            ptr = self.normalize(x, l)
        for layer in range(self.h_depth):
            ptr = getattr(self, f"linear_{layer}")(ptr)
            ptr = getattr(self, f"act_{layer}")(ptr)
            ptr = getattr(self, f"bn_{layer}")(ptr)
            ptr = getattr(self, f"dropout_{layer}")(ptr)
        loc = self.loc(ptr)
        std = F.softplus(self.std_lin(ptr)) + EPS
        print('loc',loc)
        print('std',std)
        return D.Normal(loc, std), l
#out[1] is l here


In [4]:
class NBDataEncoder(DataEncoder):

    r"""
    Data encoder for negative binomial data

    Parameters
    ----------
    in_features
        Input dimensionality
    out_features
        Output dimensionality
    h_depth
        Hidden layer depth
    h_dim
        Hidden layer dimensionality
    dropout
        Dropout rate
    """

    TOTAL_COUNT = 1e4

    def compute_l(self, x: torch.Tensor) -> torch.Tensor:
        return x.sum(dim=1, keepdim=True)

    def normalize(
            self, x: torch.Tensor, l: torch.Tensor
    ) -> torch.Tensor:
        return (x * (self.TOTAL_COUNT / l)).log1p()



In [37]:
torch.randn(5,4)

tensor([[ 1.1005,  1.0292,  0.1959, -0.6884],
        [ 1.0902, -0.8609, -0.2302, -1.1643],
        [ 0.8843, -0.3594, -0.5008, -0.3547],
        [-0.6055,  1.2214, -0.3397, -0.0994],
        [ 0.8620, -2.2344,  0.2287,  1.4481]])

In [51]:
x=torch.tensor([[0.2, 0.5, 0.8], #1.5     for graph encoder each vertices have 3 features 
      [0.3, 0.7, 0.4], # 1.4
      [0.6, 0.1, 0.9], # 1.6
      [0.9, 0.2, 0.3]])# 1.4

# if alternative x has non-zero elements
#  xrep=torch.tensor([
#        [0.6, 0.1, 0.9],
#        [0.9, 0.2, 0.3]])
xrep=torch.tensor([])
xrep=torch.abs(torch.randn(10,4))
encoder=NBDataEncoder(4,2)# Dataencoder: from 4 features to 2 features(in z)
inputx=torch.abs(torch.randn(10,4)) # number of cell N=10,number of genes(vetices) |v1|=4,m=2

u,l=encoder.forward(inputx,xrep)


loc tensor([[-0.2817,  0.3328],
        [ 0.7076, -0.1204],
        [-0.8935,  0.4211],
        [ 1.2391, -0.2269],
        [-0.7871, -0.5119],
        [ 0.3025, -0.1633],
        [ 0.1342, -0.2046],
        [ 0.7545,  0.2591],
        [-0.7137, -0.9024],
        [ 0.1048, -0.1237]], grad_fn=<AddmmBackward0>)
std tensor([[0.8224, 1.2542],
        [0.2934, 0.6911],
        [1.4690, 1.1622],
        [0.4656, 0.4686],
        [0.7069, 0.6685],
        [1.1668, 0.8798],
        [0.5931, 0.5736],
        [0.6859, 0.5418],
        [1.0748, 0.7364],
        [0.6561, 0.8459]], grad_fn=<AddBackward0>)


In [6]:
# xrep=torch.tensor([])
encoder=NBDataEncoder(3,2)# repeat above 
u,l=encoder.forward(x,xrep)
print("u",u)

loc tensor([[-0.4508, -0.8174],
        [-0.2727, -0.7954],
        [ 1.3175,  0.4944],
        [-0.1101,  1.1146]], grad_fn=<AddmmBackward0>)
std tensor([[1.0225, 1.2590],
        [1.5359, 0.5046],
        [0.3934, 1.0585],
        [0.5192, 0.2525]], grad_fn=<AddBackward0>)
u Normal(loc: torch.Size([4, 2]), scale: torch.Size([4, 2]))


In [7]:
print(l)# sum of x 
print(u.loc)
print(u.scale)

tensor([[1.5000],
        [1.4000],
        [1.6000],
        [1.4000]])
tensor([[-0.4508, -0.8174],
        [-0.2727, -0.7954],
        [ 1.3175,  0.4944],
        [-0.1101,  1.1146]], grad_fn=<AddmmBackward0>)
tensor([[1.0225, 1.2590],
        [1.5359, 0.5046],
        [0.3934, 1.0585],
        [0.5192, 0.2525]], grad_fn=<AddBackward0>)


from sc.py get sc.GraphEncoder 

In [8]:

# class GraphEncoder0(torch.nn.Module):

#     r"""
#     Abstract graph encoder
#     """

#     @abstractmethod
#     def forward(
#             self, eidx: torch.Tensor, enorm: torch.Tensor, esgn: torch.Tensor
#     ) -> D.Distribution:
#         r"""
#         Encode graph to vertex latent distribution

#         Parameters
#         ----------
#         eidx
#             Vertex indices of edges (:math:`2 \times n_{edges}`)
#         enorm
#             Normalized weight of edges (:math:`n_{edges}`)
#         esgn
#             Sign of edges (:math:`n_{edges}`)

#         Returns
#         -------
#         v
#             Vertex latent distribution
#             (:math:`n_{vertices} \times n_{features}`)
#         """
#         raise NotImplementedError  # pragma: no cover



class GraphEncoder(glue.GraphEncoder):

    r"""
    Graph encoder

    Parameters
    ----------
    vnum
        Number of vertices
    out_features
        Output dimensionality
    """

    def __init__(
            self, vnum: int, out_features: int
    ) -> None:
        super().__init__()
        self.vrepr = torch.nn.Parameter(torch.zeros(vnum, out_features))
        self.conv = GraphConv()#图卷积
        self.loc = torch.nn.Linear(out_features, out_features) #two fully connected layer
        self.std_lin = torch.nn.Linear(out_features, out_features)

    def forward(# output is v
            self, eidx: torch.Tensor, enorm: torch.Tensor, esgn: torch.Tensor
    ) -> D.Normal:
        ptr = self.conv(self.vrepr, eidx, enorm, esgn) #output of graph convolution
        loc = self.loc(ptr) # for mean and variance using two linear layer
        std = F.softplus(self.std_lin(ptr)) + 1e-7
        print('Normal sample: ',D.Normal(loc, std).sample_n(2,))
        print('Normal sample: ',D.Normal(loc, std).sample())
        print('//////////////')
        return D.Normal(loc, std) #output a normal distribution


In [52]:
#to get graph embedding v
#the graph has 4 vertices and 5 edges
edges =torch.tensor([
    [0, 1],  # 边1连接节点0和节点1
    [1, 2],  # 边2连接节点1和节点2
    [2, 0],  # 边3连接节点2和节点0
    [1, 3],  # 边4连接节点1和节点3
    [3, 0],  # 边5连接节点3和节点0
])
# the number of genes is size of vertices and m=2
gencoder=GraphEncoder(4,2)# 4 vertices and out_features is 2
#weight on edges 
weight=torch.tensor([[0.6,0.3,0.7,0.5,0.9]]) # toy weight
weight=torch.flatten(F.normalize(weight))
#signs on edges 
sign=torch.flatten(torch.tensor([[1,1,1,-1,1]]))
# model=GraphConv()
# model.forward(x,torch.transpose(edges,0,1),weight,sign)
# to use graph encoder to get embedding/distribution v (|v1|=4,m=2)
v=gencoder.forward(torch.transpose(edges,0,1),weight,sign)
print(v.loc)
print(v.scale)
print(v.sample())

Normal sample:  tensor([[[ 7.2776e-01,  8.3075e-01],
         [ 6.5701e-02,  6.9408e-01],
         [-5.2981e-01, -1.2566e+00],
         [-3.0500e-01,  4.1168e-04]],

        [[ 5.3128e-01,  2.6281e-01],
         [-4.7231e-01, -4.8832e-01],
         [ 5.5304e-01, -2.2238e-01],
         [ 2.3909e-01, -1.6918e-01]]])
Normal sample:  tensor([[-0.5816, -0.2778],
        [-0.3754,  0.0418],
        [-0.5096, -0.8488],
        [-0.0242, -0.9460]])
//////////////
tensor([[-0.0960, -0.0845],
        [-0.0960, -0.0845],
        [-0.0960, -0.0845],
        [-0.0960, -0.0845]], grad_fn=<AddmmBackward0>)
tensor([[0.4889, 0.5951],
        [0.4889, 0.5951],
        [0.4889, 0.5951],
        [0.4889, 0.5951]], grad_fn=<AddBackward0>)
tensor([[ 0.1959, -0.0101],
        [-0.6630,  0.1759],
        [-0.0585, -0.3054],
        [ 0.6951,  1.0880]])


In [10]:
n_sample=2 # some tests 
zv=torch.cat([v.sample((1, )).cpu() for _ in range(n_sample)
            ]).permute(1, 0, 2).numpy()
zv=v.mean.detach().cpu().numpy()
print(v.sample())
zv

tensor([[-0.8182, -0.3265],
        [ 0.2091,  0.3257],
        [-1.5657, -0.2676],
        [-0.8431, -1.1648]])


array([[-0.6640681 , -0.30737612],
       [-0.6640681 , -0.30737612],
       [-0.6640681 , -0.30737612],
       [-0.6640681 , -0.30737612]], dtype=float32)

In [41]:

class NBDataDecoder(DataDecoder):

    r"""
    Negative binomial data decoder

    Parameters
    ----------
    out_features
        Output dimensionality
    n_batches
        Number of batches
    """

    def __init__(self, out_features: int, n_batches: int = 1) -> None:
        super().__init__(out_features, n_batches=n_batches)
        self.scale_lin = torch.nn.Parameter(torch.zeros(n_batches, out_features))
        self.bias = torch.nn.Parameter(torch.zeros(n_batches, out_features))
        self.log_theta = torch.nn.Parameter(torch.zeros(n_batches, out_features))

    def forward(
            self, u: torch.Tensor, v: torch.Tensor,
            b: torch.Tensor, l: torch.Tensor
    ) -> D.NegativeBinomial:
        scale = F.softplus(self.scale_lin[b])
        print(u.shape,v.t().shape,scale.shape)
        print(scale, self.scale_lin)
        #torch.Size([128, 50]) torch.Size([50, 800]) torch.Size([128, 800])
        logit_mu = scale * (u @ v.t()) + self.bias[b]
        mu = F.softmax(logit_mu, dim=1) * l
        log_theta = self.log_theta[b]
        return D.NegativeBinomial(
            log_theta.exp(),
            logits=(mu + EPS).log() - log_theta
        )
    

In [47]:
#decoder need graph embedding v
from scglue.models import data
decoder=NBDataDecoder(4)# out_feature is 4
b=torch.tensor([0,0,0,0,0,0,0,0,0,0])# of size 10
    
decoder.forward(u.sample(),v.sample(),b,l).mean #size 10,4


torch.Size([10, 2]) torch.Size([2, 4]) torch.Size([10, 4])
tensor([[0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.6931]], grad_fn=<SoftplusBackward0>) Parameter containing:
tensor([[0., 0., 0., 0.]], requires_grad=True)


tensor([[0.3961, 0.3183, 0.4803, 0.5708],
        [0.2632, 0.0734, 0.6305, 1.5664],
        [0.4310, 1.1609, 0.2773, 0.1500],
        [1.4730, 1.1394, 0.5624, 0.4345],
        [0.4045, 0.8802, 0.2500, 0.1465],
        [1.7561, 0.5880, 0.9617, 1.2429],
        [0.2681, 1.0977, 0.2508, 0.1300],
        [0.5110, 3.1383, 0.9444, 0.5316],
        [1.3598, 2.4380, 0.8561, 0.5510],
        [1.3997, 0.6251, 1.2062, 1.6334]], grad_fn=<MulBackward0>)

In [48]:
inputx

tensor([[1.1177, 0.2614, 0.3580, 0.0284],
        [0.1492, 1.2228, 0.1198, 1.0416],
        [0.4966, 0.5017, 0.6235, 0.3974],
        [1.4366, 1.0217, 0.5163, 0.6349],
        [0.6009, 0.2623, 0.6252, 0.1928],
        [1.3275, 1.4076, 1.3831, 0.4304],
        [0.2682, 0.2548, 0.8072, 0.4164],
        [0.9626, 1.1145, 2.9090, 0.1393],
        [0.8406, 1.9030, 1.7614, 0.6999],
        [1.7558, 1.8256, 0.1858, 1.0971]])

In [ ]:
v.sample().t().shape

In [ ]:
scl=torch.nn.Parameter(torch.rand(1, 7))
scl.shape
b = torch.tensor([1, 3, 4]) 

In [4]:
class SCGLUEModel(Model):

    r"""
    GLUE model for single-cell multi-omics data integration

    Parameters
    ----------
    adatas
        Datasets (indexed by modality name)
    vertices
        Guidance graph vertices (must cover feature names in all modalities)
    latent_dim
        Latent dimensionality
    h_depth
        Hidden layer depth for encoder and discriminator
    h_dim
        Hidden layer dimensionality for encoder and discriminator
    dropout
        Dropout rate
    shared_batches
        Whether the same batches are shared across modalities
    random_seed
        Random seed
    """

    NET_TYPE = SCGLUE
    TRAINER_TYPE = SCGLUETrainer

    GRAPH_BATCHES: int = 32  # Number of graph batches in each graph epoch
    ALIGN_BURNIN_PRG: float = 8.0  # Effective optimization progress of align_burnin (learning rate * iterations)
    MAX_EPOCHS_PRG: float = 48.0  # Effective optimization progress of max_epochs (learning rate * iterations)
    PATIENCE_PRG: float = 4.0  # Effective optimization progress of patience (learning rate * iterations)
    REDUCE_LR_PATIENCE_PRG: float = 2.0  # Effective optimization progress of reduce_lr_patience (learning rate * iterations)

    def __init__(
            self, adatas: Mapping[str, AnnData],
            vertices: List[str], latent_dim: int = 50,
            h_depth: int = 2, h_dim: int = 256,
            dropout: float = 0.2, shared_batches: bool = False,
            random_seed: int = 0
    ) -> None:
        # print(vertices)
        self.vertices = pd.Index(vertices)
        # print('vertices: ',self.vertices)
        self.random_seed = random_seed
        torch.manual_seed(self.random_seed)

        g2v = sc.GraphEncoder(self.vertices.size, latent_dim)# latent_dim:50
        v2g = sc.GraphDecoder()# modalities: rna: batches, features
        self.modalities, idx, x2u, u2x, all_ct = {}, {}, {}, {}, set()
        for k, adata in adatas.items(): #k is keys 'rna' or 'atac' and adata is value
            if config.ANNDATA_KEY not in adata.uns:
                raise ValueError(
                    f"The '{k}' dataset has not been configured. "
                    f"Please call `configure_dataset` first!"
                )
            data_config = copy.deepcopy(adata.uns[config.ANNDATA_KEY])# by config.ANNDATA_KEY
            print('data_config', data_config)    #'prob_model': 'NB', 'use_highly_variable': True, 'features':
            if data_config["rep_dim"] and data_config["rep_dim"] < latent_dim:
                self.logger.warning(
                    "It is recommended that `use_rep` dimensionality "
                    "be equal or larger than `latent_dim`."
                ) # store vertex feature index into idx classified by modalities
            idx[k] = self.vertices.get_indexer(data_config["features"]).astype(np.int64)#rna/atac's gene index
            if idx[k].min() < 0:
                raise ValueError("Not all modality features exist in the graph!")
            idx[k] = torch.as_tensor(idx[k])#_Encoder_map all from sc.py
            x2u[k] = _ENCODER_MAP[data_config["prob_model"]](
                data_config["rep_dim"] or len(data_config["features"]), latent_dim,
                h_depth=h_depth, h_dim=h_dim, dropout=dropout
            )
            data_config["batches"] = pd.Index([]) if data_config["batches"] is None \
                else pd.Index(data_config["batches"]) #data_config contain 'batches', 'rep_dim','features'
            u2x[k] = _DECODER_MAP[data_config["prob_model"]](#data decoder
                len(data_config["features"]),
                n_batches=max(data_config["batches"].size, 1)
            )
            all_ct = all_ct.union(
                set() if data_config["cell_types"] is None
                else data_config["cell_types"]
            )
            self.modalities[k] = data_config
            # print('modalities', self.modalities)#{'rna': {'prob_model': 'NB', 'use_highly_variable': True, 'features':

        all_ct = pd.Index(all_ct).sort_values()
        for modality in self.modalities.values():
            modality["cell_types"] = all_ct
        if shared_batches:
            all_batches = [modality["batches"] for modality in self.modalities.values()]
            ref_batch = all_batches[0]
            for batches in all_batches:
                if not np.array_equal(batches, ref_batch):
                    raise RuntimeError("Batches must match when using `shared_batches`!")
            du_n_batches = ref_batch.size
        else:
            du_n_batches = 0
        du = sc.Discriminator(
            latent_dim, len(self.modalities), n_batches=du_n_batches,
            h_depth=h_depth, h_dim=h_dim, dropout=dropout
        )
        prior = sc.Prior()
        super().__init__( #base object stored as derived class
            g2v, v2g, x2u, u2x, idx, du, prior,
            u2c=None if all_ct.empty else sc.Classifier(latent_dim, all_ct.size)
        )

NameError: name 'Model' is not defined

In [2]:
import torch
import torch.nn as nn
from itertools import chain

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear = nn.Linear(10, 5)
        self.register_buffer('buffer1', torch.tensor([1.0, 2.0, 3.0]))
    
    def forward(self, x):
        return self.linear(x)

model = MyModel()

# 遍历目标模型的参数和命名缓冲区，并打印名称和张量
for name, tensor in chain(model.named_parameters(), model.named_buffers()):
    print(name)
#     print(tensor)

linear.weight
linear.bias
buffer1


In [5]:
for name, tensor in chain(model.named_parameters(), model.named_buffers()):
    print(tensor)
    if isinstance(name, torch.nn.Parameter):# if t is an instance of nn.Parameter
        t = name.data 
        print("t",t)
    print(tensor==tensor.data)
    

Parameter containing:
tensor([[-0.2956,  0.2632, -0.0399, -0.0600, -0.2162,  0.1683,  0.1146,  0.2986,
         -0.0525, -0.0917],
        [ 0.2347,  0.0077,  0.2026, -0.1359,  0.2513,  0.2200,  0.2738,  0.3159,
         -0.2503,  0.0249],
        [-0.1529,  0.1079, -0.0804, -0.1616,  0.1510,  0.0086,  0.0306, -0.0593,
         -0.1751,  0.0961],
        [ 0.0552,  0.1704,  0.3098,  0.0919, -0.2823, -0.1996,  0.2215,  0.2133,
         -0.2431,  0.1442],
        [-0.0587, -0.0956, -0.0817,  0.2263, -0.3023,  0.0004,  0.0160, -0.2632,
         -0.1567,  0.1490]], requires_grad=True)
tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
Parameter containing:
tensor([ 0.2092, -0.1887,  0.2704,  0.0

In [1]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


In [5]:
from ignite.engine import Engine, Events

def update_fn(engine, batch):
    # 在此处执行训练逻辑
#     print('training process')
    return 2

# 创建训练引擎
trainer = Engine(update_fn)

@trainer.on(Events.EPOCH_COMPLETED)
def log_epoch(engine):
    epoch = engine.state.epoch
    print(f"Training Epoch: {epoch}")

# 执行训练过程
trainer.run(train_dataloader, max_epochs=10)

Training Epoch: 1
Training Epoch: 2
Training Epoch: 3
Training Epoch: 4
Training Epoch: 5
Training Epoch: 6
Training Epoch: 7
Training Epoch: 8
Training Epoch: 9
Training Epoch: 10


State:
	iteration: 7820
	epoch: 10
	epoch_length: 782
	max_epochs: 10
	output: 2
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>